# Lab 5: Naive Bayes and Random Forests
In this lab we'll learn to use [GridSearchCV] and take a closer look the [Naive Bayes](http://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html#sklearn.naive_bayes.MultinomialNB) and [Random Forests](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) classifiers.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import train_test_split
%matplotlib inline

/usr/local/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# Naive Bayes
Naive Bayes widely known for being effective for spam email classification. We'll try our hand at applying NB on the [Spam email dataset](https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection) from the UCI data repository. Download the dataset and move it to your lab data directory.

In [2]:
fname = 'data/SMSSpamCollection'
df = pd.read_table(fname, names=['class', 'text'])
df.head()

,class,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


## Multinomial Naive Bayes
Given collection of emails $d^{(1)}, d^{(2)}, ..., d^{(N)}$ with corresponding labels $y^{(i)}$, we first need to extract features from each document. We'll do this by computing word counts of each word in each document. This is commonly referred to as a [bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model) representation.

After doing some preprocessing of our data, we'll have: $\{(x^{(i)}, y^{(i)})\}_{i=1}^N$, where:
* $x^{(i)} \in R^d$ is a word count feature vector for document $i$, and $d$ is the size of the vocabulary. For the purposes of this lab, we define our vocabulary to be the set of words that appear in our dataset of documents.
$$ x^{(i)} = (\theta^{(i)}_1, \theta^{(i)}_2, ..., \theta^{(i)}_d)$$
$$\theta^{(i)}_k = \text{ the number of times word $k$ appears in document $i$}$$


* $y^{(i)} \in \{0, 1\}$ is its class, where $0$ denotes spam, and $1$ denotes real email.

First we need to compute word count feature vectors. We can do this manually like so:

In [3]:
import string

def get_all_words(docs):
    '''
    docs: pandas.Series of strings
    Returns: set of words(strings)
    '''
    # Compute all the words in the docs
    words = set()
    for d in docs:
        # this will strip punctuation
        # ref: https://stackoverflow.com/questions/265960/best-way-to-strip-punctuation-from-a-string-in-python
        d = d.translate(string.punctuation)
        words = words.union(set(d.split()))
    return words

def make_word_cnt_feats(docs, vocabulary=None):
    '''
    docs: iterable(ie: list or pandas.Series) of strings
    vocabulary: set of words(strings) that appear in docs
    Returns: numpy matrix of the word count features
    '''
    if vocabulary is None:
        vocabulary = get_all_words(docs)
    words = sorted(vocabulary)
    word_dict = {w: idx for idx, w in enumerate(words)}
    
    # feature matrix will be of size n x (size of vocabulary + 1)
    # add 1 to account for words that dont appear in the vocabulary to avoid random issues
    vocab_size = len(words)
    word_cnt_features = np.zeros((len(docs), len(words) + 1))
    
    for idx in range(len(docs)):
        doc = docs[idx]
        doc = doc.translate(string.punctuation)
        words = doc.split()
        for w in words:
            word_idx = word_dict.get(w, vocab_size)
            word_cnt_features[idx, word_idx] += 1
            
    return word_cnt_features

In [4]:
vocabulary = get_all_words(df['text'])
feature_matrix = make_word_cnt_feats(df['text'], vocabulary)
print('Feature matrix shape: {}'.format(feature_matrix.shape))

Feature matrix shape: (5572, 15693)


It turns out that sklearn actually has a built-in method to do this kind of word count featurization! (But of course, it's good to get your hands dirty with manual featurization from time to time to understand all the ingredients of these algorithms). See sklearn's [sklearn.feature_extraction.text.CountVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) function for more details.                                                                 

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
count_vectorizer = CountVectorizer()
word_cnt_feats = count_vectorizer.fit_transform(df['text'])
word_cnt_feats
# looks like this gives us fewer columns, which means we missed
# a bunch of unwanted terms(maybe words with numbers?)

<5572x8713 sparse matrix of type '<class 'numpy.int64'>'
	with 74169 stored elements in Compressed Sparse Row format>

[CountVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) has a bunch of useful functions that you should checkout. The important ones that we'll use are:
* [fit](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html#sklearn.feature_extraction.text.CountVectorizer.fit): this learns the vocabulary of the input collection of strings/"documents"
* [transform](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html#sklearn.feature_extraction.text.CountVectorizer.transform): given a collection of documents, this produces the word count features for the documents
* fit_transform: calls fit, then transform

Note that the word count feature matrix(aka document-term matrix) is in sparse format. This is not an issue for the sklearn models. But if you want to expand them to non-sparse numpy arrays, use the toarray function. Note that this can be problematic for bigger datasets! We can somewhat get around this b/c this dataset is small enough that we can fully expand a 5k by 8k matrix(since it's sparse enough).

In [6]:
X = word_cnt_feats.toarray()
Y = (df['class'] == 'ham').values
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.4)

In [7]:
Y.count()

AttributeError: 'numpy.ndarray' object has no attribute 'count'

Now that we have the data in the document-term matrix format, we can use the Naive Bayes classifier!

In [8]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
nb = MultinomialNB(alpha=1)
nb.fit(X_train, y_train)
preds = nb.predict(X_test)

In [9]:
print('Accuracy: {:.3f}'.format(accuracy_score(preds, y_test)))
print('F1 Score: {:.3f}'.format(f1_score(preds, y_test)))
print('ROC AUC Score: {:.3f}'.format(roc_auc_score(preds, y_test)))

Accuracy: 0.978
F1 Score: 0.987
ROC AUC Score: 0.947


## Naive Bayes Tasks
Now it's your turn to use it on the Naive Bayes model. Pick an evaluation metric to use for cross validation like F1 score, ROC AUC score, etc. In the spam classification setting, do you think it makes sense to optimize for higher precision? recall? f1 score?

0) Pick an evaluation metric.

1) Note that the code above fixed alpha to 1. But as we've learned in class and last week's lab, we should use cross validation for any kind of parameter search. Use [KFold](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html) to do the cross validation over 5 folds to find the alpha that works best on the training set.

In [22]:
from sklearn.model_selection import KFold
import pdb
alphas = [1, 5, 10, 25, 100] # you should vary these 
kf = KFold(n_splits=5)

def naive_bayes_kfold(X_train, y_train, alphas, kf):
    cv_split_results = {} # store your cross validation results

    for fold_num, (train_idx, test_idx) in enumerate(kf.split(X_train)):
        x_split_train, x_split_test = X_train[train_idx], X_train[test_idx]
        y_split_train, y_split_test = y_train[train_idx], y_train[test_idx]

        for a in alphas:
            nb = MultinomialNB(alpha=a)
            nb.fit(x_split_train, y_split_train)
            preds = nb.predict(x_split_test)
            f1 = f1_score(preds, y_split_test)
            cv_split_results[fold_num,a] =  cv_split_results.get(a, 0) + f1
            
    return cv_split_results

naive_bayes_kfold(X_train, y_train, alphas, kf)

{(0, 1): 0.9896551724137931,
 (0, 5): 0.9788672865595943,
 (0, 10): 0.9731993299832497,
 (0, 25): 0.9555921052631579,
 (0, 100): 0.9318364073777066,
 (1, 1): 0.9913793103448276,
 (1, 5): 0.9690376569037658,
 (1, 10): 0.9586776859504132,
 (1, 25): 0.9392712550607287,
 (1, 100): 0.9309791332263242,
 (2, 1): 0.9914383561643836,
 (2, 5): 0.9806560134566863,
 (2, 10): 0.96843853820598,
 (2, 25): 0.9502852485737571,
 (2, 100): 0.9350441058540497,
 (3, 1): 0.9929701230228472,
 (3, 5): 0.9801894918173988,
 (3, 10): 0.9685106382978723,
 (3, 25): 0.9443983402489626,
 (3, 100): 0.9229521492295215,
 (4, 1): 0.9868536371603857,
 (4, 5): 0.9734816082121472,
 (4, 10): 0.9538977367979882,
 (4, 25): 0.9366255144032921,
 (4, 100): 0.9229521492295215}

2) Now that you've found the optimal alpha using cross validation over the training set, evaluate the model on the test set.

In [24]:
# alpha = 1 performs best
nb = MultinomialNB(alpha=1)
nb.fit(X_train, y_train)
preds = nb.predict(X_test)
f1 = f1_score(preds, y_test)
f1

0.9873678783191544

Further Questions:
* Naive Bayes has a seemingly stringent independence assumption. Do you think the variables in this email classification problem(word count vectors) are really independent? Does this matter? Why/why not?

## Random Forests
Two weeks ago, we played around with decision trees. In general, practictioners almost never use decision trees over Random Forests. Recall from lecture, that Random Forests takes some number of subsamples of the data:
$S_1, S_2, ..., S_k \subseteq \{(x^{(i)}, y^{(i)}\}_{i=0}^N$. On each subsample of the data, $S_i$, the Random Forests algorithm trains a decision tree on $S_i$.

At classification time, when we get a datapoint $x$, the Random Forests classifier runs $x$ through decision trees $D_1, D_2, ..., D_k$, which produces a "probability" estimate of each class:

$$P(y = 1 | x) = \frac{\text{number of decision trees that classified $x$ as 1}}{k}$$

$$P(y = 0 | x) = \frac{\text{number of decision trees that classified $x$ as 1}}{k}$$

Then from there, we get a class prediction by picking $y = 1$ if $P(y = 1 | x) > T$, for some threshold probability $T$.


Instead of using the spam dataset to try out Random Forests, we'll re-use the credit loan data from Assignment 2.

In [ ]:
# load data

Do whatever data cleaning, data imputing, etc you need to do.

In [ ]:
# clean data

Split the data into train/test sets.

In [ ]:
# split data

## Random Forest Parameters

The Random Forest parameters are more or less the same as those for Decision Trees. The only additional parameter is n_estimators, which is the number of decision trees to grow for the algorithm.
* n_estimators : integer, the number of trees in the forest.
* criterion : string,  “gini” or “entropy” 
* max_features : int, float, string or None, the number of features to consider when looking for the best split
* max_depth : integer, the maximum depth of the tree
* min_samples_split : int, float, the minimum number of samples required to split an internal node:
* min_samples_leaf : int, float, optional (default=1), the minimum number of samples required to be at a leaf node:
* min_weight_fraction_leaf : float, optional 
* max_leaf_nodes : int or None, optional (default=None)

0) What do you think will be the effect of increasing the n\_estimators parameter? If n_estimators = 1, is this any different to using a decision tree classifier(assuming you set all the other parameters like min_sample_split, max_depth, etc the same way in both cases)?

1) Pick an evaluation metric.

2) Pick a few values for each of the random forest parameters for cross validation.

In [ ]:
# param settings to try out

In last weeks lab, we saw how to grid search over all possible model parameters and cross validation folds. To refresh your memory, below is a slightly updated version of that code.

In [ ]:
# Example of a manual grid search for LogisticRegression on random data
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
def sample_manual_grid_search(n_samples, n_feats, splits):
    # make the fake data
    X = np.random.randint(0, 10, (n_samples, n_feats))
    Y = (np.random.random(n_samples) > 0.5) # random true/false
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.4)
    
    # params to search over
    penalties = ['l1', 'l2']
    c_vals = [10**i for i in range(-2, 3)]

    kf = KFold(n_splits=splits)
    rows = [] # store dicts of the cross validation results for each fold
    for p in penalties:
        for c in c_vals:

            fold_dict = {
                    # NOTE: these key values must be exactly as they're spelled in
                    # the sklearn model!
                    'penalty': p,
                    'C': c,
            }
            fold_results = {}
            for fold_num, (train_idx, test_idx) in enumerate(kf.split(X_train, y_train)):
                x_split_train, x_split_test = X_train[train_idx], X_train[test_idx]
                y_split_train, y_split_test = y_train[train_idx], y_train[test_idx]
                    
                model = LogisticRegression(**fold_dict)
                model.fit(x_split_train, y_split_train)
                pred_probs = model.predict_proba(x_split_test)
                loss = roc_auc_score(y_split_test, pred_probs[:, 0])
                fold_results['fold_{}'.format(fold_num+1)] = loss
                    
                # now that we have all the fold results, add them to the dataframe
            fold_dict.update(fold_results)
            fold_dict['avg_score'] = np.mean(fold_results.values())
            rows.append(fold_dict)

    res_df = pd.DataFrame(rows)
    return res_df


In [ ]:
cv_results = sample_manual_grid_search(n_samples=100, n_feats=20, splits=5)
print("Grid Search Results for sample data with Logistic Regression:")
cv_results.head()

3) Do the same kind of grid search cross validation for Random Forests classifier. Which set of parameters gets the best average score over all folds?

In [ ]:
# Your grid search CV for RandomForests


Writing 8x nested for-loops to search over all parameters can be a bit cumbersome. Luckily, sklearn has a very nice [GridSearchCV class](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) that can do all this grid searching and cross validating for us!


## Grid Search Cross Validation
GridSearchCV requires:
* an estimator object(IE: LogisticRegression(), SVC(), KNeighborsClassifier(), etc)
* param_grid: a dictionary mapping parameter names to lists of values to search over. We'll see an example below
* scoring: a string. See the valid inputs here: http://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
* n_jobs: number of jobs to run in parallel to speed up the computation. GridSearchCV can take a long time if you have a lot of different parameter combinations to try out. So it's important to set this parameter if your laptop/desktop has a lot of cores that you want to take advantage of.
* cv: int, number of cross validation folds to use
* refit: boolean, if true, at the end of the Grid Search, GridSearchCV will refit the entire data with a model using the optimal parameters(you can then access this with the GridSearchCV.best\_parameters_ attribute).

Here's an example of how we'd use it for some random data:

In [ ]:
# Example of how to use GridSearchCV
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

def sample_cross_val_param_search():
    # parameters to search over
    penalties = ['l1', 'l2']
    c_vals = [10**i for i in range(-2, 3)]
    param_grid = {'penalty': penalties, 'C': c_vals}
    
    # make some fake data
    n_samples = 1000
    feats = 10
    x_train = np.random.random((n_samples, feats))
    y_train = (np.random.random(n_samples) > 0.5) # randomly generate True/False labels
    
    grid = GridSearchCV(LogisticRegression(), param_grid, 'f1', cv=5)
    grid.fit(x_train, y_train)
    return grid
grid = sample_cross_val_param_search()
results = pd.DataFrame(grid.cv_results_)
print("GridSearchCV results:")
results.head(3)

In [ ]:
print('Best estimator: {}'.format(grid.best_estimator_))
print('Best Parameters: {}'.format(grid.best_params_))

4) Use GridSearchCV to do the same parameter search you did above. What is the best performing set of parameters? Do they match up with what you had above?

In [ ]:
# CODE


5) As mentioned earlier, GridSearchCV.best\_estimator\_ will contain a model with parameters set to the best performing set of parameters fitted on the entire training set. Use this best estimator on the test set.

In [ ]:
# CODE


6) Random Forests has a [feature importances](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html#sklearn.ensemble.RandomForestClassifier.feature_importances_) attribute. Which features are the most important?

In [ ]:
# code

4) Rerun the classifier with a different scoring metric. Do the feature importances change? Which features are most important now?

In [ ]:
# code

## References
* https://etav.github.io/projects/spam_message_classifier_naive_bayes.html
* http://scikit-learn.org/stable/modules/naive_bayes.html
* http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html